In [1]:
from json import dumps, loads
from pathlib import Path
import sys

import numpy as np
import pandas as pd
import scanpy as sc
from scipy.sparse import csr_matrix
from IPython.display import display

p_link = Path("/public/workspace/licanchengup/link")
p_test = p_link.joinpath('test')
p_disease = p_link.joinpath('disease')
display(p_disease)

PosixPath('/public/workspace/licanchengup/link/disease')

In [2]:
sys.path.append(str(p_test))

In [3]:
from func import h5ad_to_mtx, group_agg

Help on function get_counts_from_adatas in module csMAHN.utils.preprocess:

get_counts_from_adatas(adatas)
    Assum adatas has been normalized
    Get a pair of dataframes from a pair adatas
    
    :param adatas: a tuple of reference adata and query adata
    :return: a tuple of reference raw counts dataframe and query raw counts dataframe

Help on function get_counts_from_adatas in module func:

get_counts_from_adatas(adatas)
    补丁
    version = 0.0.3


[func]----------------------------------
group_agg
rm_rf
get_path_varmap
get_type_counts_info		aligned
aligned_type			aligned
test_came_and_csMAHN
h5ad_to_mtx
load_adata

[result tools]--------------------------
get_test_result_df
show_umap
get_res_obs
get_publi_head

[result tools][re plot umap]-----------
get_df_color			Convenient
> [dssp umap]
export_legend
dssp_plot_umap_legend_sp	Convenient
dssp_plot_umap_legend_cell_type	Convenient
dssp_plot_umap
dssp_get_umap_name
dssp_plot_save_umap		Convenient
> [umap]
get_default_df_color

In [4]:
df_path = pd.DataFrame({'path': [i for i in Path(
    'GSE145286_RAW').iterdir() if i.match('*sample*')]})
df_path['name'] = df_path['path'].apply(lambda x: x.stem)
df_path = df_path.join(df_path['name'].str.extract(
    "(?P<gse_id>GSM\\d+)_(?P<type>\\w+)_(?P<sample>sample\\d)\\."))
df_path = df_path.pivot(index='gse_id', columns='type', values='path')
df_path

type,barcodes,counts,genes
gse_id,,,
GSM4314258,GSE145286_RAW/GSM4314258_barcodes_sample1.txt.gz,GSE145286_RAW/GSM4314258_counts_sample1.mtx.gz,GSE145286_RAW/GSM4314258_genes_sample1.txt.gz
GSM4314259,GSE145286_RAW/GSM4314259_barcodes_sample2.txt.gz,GSE145286_RAW/GSM4314259_counts_sample2.mtx.gz,GSE145286_RAW/GSM4314259_genes_sample2.txt.gz
GSM4314260,GSE145286_RAW/GSM4314260_barcodes_sample3.txt.gz,GSE145286_RAW/GSM4314260_counts_sample3.mtx.gz,GSE145286_RAW/GSM4314260_genes_sample3.txt.gz
GSM4314261,GSE145286_RAW/GSM4314261_barcodes_sample4.txt.gz,GSE145286_RAW/GSM4314261_counts_sample4.mtx.gz,GSE145286_RAW/GSM4314261_genes_sample4.txt.gz
GSM4314262,GSE145286_RAW/GSM4314262_barcodes_sample5.txt.gz,GSE145286_RAW/GSM4314262_counts_sample5.mtx.gz,GSE145286_RAW/GSM4314262_genes_sample5.txt.gz
GSM4314263,GSE145286_RAW/GSM4314263_barcodes_sample6.txt.gz,GSE145286_RAW/GSM4314263_counts_sample6.mtx.gz,GSE145286_RAW/GSM4314263_genes_sample6.txt.gz


In [5]:
for i, row in df_path.iterrows():
    row
row

type
barcodes    GSE145286_RAW/GSM4314263_barcodes_sample6.txt.gz
counts        GSE145286_RAW/GSM4314263_counts_sample6.mtx.gz
genes          GSE145286_RAW/GSM4314263_genes_sample6.txt.gz
Name: GSM4314263, dtype: object

In [6]:
def load_adata_frome_mtx(row, show=True):
    adata = sc.read_mtx(row['counts']).T
    df_obs = pd.read_csv(row['barcodes'], index_col=0, header=None)
    df_obs.index.name = None
    df_obs['gse_id'] = row.name

    df_var = pd.read_csv(row['genes'], index_col=0, header=None)
    df_var.index.name = None
    adata.obs = df_obs
    adata.var = df_var

    display(
        row.name.ljust(50, '-'),
        adata,
        adata.shape,
        adata.obs.head(),
        adata.obs.shape,
        adata.var.head(),
        adata.var.shape) if show else None
    return adata


load_adata_frome_mtx(row)

'GSM4314263----------------------------------------'

AnnData object with n_obs × n_vars = 2109 × 31017
    obs: 'gse_id'

(2109, 31017)

,gse_id
AAACCTGAGCTTTGGT_BRI070,GSM4314263
AAACCTGGTGCTGTAT_BRI070,GSM4314263
AAACCTGTCATGTGGT_BRI070,GSM4314263
AAACCTGTCCTTTACA_BRI070,GSM4314263
AAACGGGAGTGTTGAA_BRI070,GSM4314263


(2109, 1)

""
0610005C13Rik
0610006L08Rik
0610009B22Rik
0610009O20Rik
0610010F05Rik


(31017, 0)

AnnData object with n_obs × n_vars = 2109 × 31017
    obs: 'gse_id'

In [7]:
data = {i: load_adata_frome_mtx(row, show=False)
        for i, row in df_path.iterrows()
        }
adata = sc.concat(data)
adata.obs.head()

,gse_id
AAACCTGAGAACAATC_BRI203,GSM4314258
AAACCTGAGCTCCCAG_BRI203,GSM4314258
AAACCTGAGGTGCTAG_BRI203,GSM4314258
AAACCTGCAGGTTTCA_BRI203,GSM4314258
AAACGGGAGGATGTAT_BRI203,GSM4314258


In [8]:
# ','.join(df_meta.columns)
# sampleID,status,label,cell_type,cell_subtype,nUMI,nGene,percent_mito,pseudotime
df_meta = pd.read_csv("SCP469/metadata/metaData.txt",
                      sep='\t',
                      index_col=0).loc[:,
                                       'sampleID,status,label,cell_type,cell_subtype'.split(',')]
# df_meta
display(df_meta.head(2), adata.obs.shape, adata.obs.join(df_meta).shape)

,sampleID,status,label,cell_type,cell_subtype
NAME,,,,,
TYPE,group,group,group,group,group
AAACCTGAGACAAAGG_BWH075,donor2,ra,tissue,Lining Fibroblasts,Lining Fibroblasts


(18491, 1)

(18491, 6)

In [9]:
df_umap = pd.DataFrame(loads(
    Path('SCP469/UMAP mouse.json').read_text())['data'])
df_umap.index = df_umap['cells'].to_numpy()
df_umap = df_umap.loc[:, 'x,y'.split(',')].rename(
    columns={'x': 'umap_x', 'y': 'umap_y'})
df_umap.head()

,umap_x,umap_y
AACCGCGTCCGAATGT_BRI203,2.419474,3.522746
GGGAATGAGGGTTCCC_BRI070,9.546767,2.541875
GTGCATATCGGACAAG_BRI069,-5.522621,3.016629
GTATTCTGTCTCCCTA_BRI070,6.104962,-3.867551
GCGGGTTCACTATCTT_BRI203,-5.607356,5.442302


In [10]:
df_umap = df_umap.join(df_meta)
df_umap = df_umap.loc[:,
                      'sampleID,status,label,cell_type,umap_x,umap_y'.split(',')]
display(df_umap['status'].value_counts(), df_umap['label'].value_counts())

df_umap['status'] = df_umap['status'].map({'ctrl': 'Healty',
 'case': 'RA'})
df_umap['label'] = df_umap['label'].map({k: v for k, v in zip(
    'WT,KO,ISO,N3'.split(','), 'WT,Notch3−/−,ISO,anti-NRR3'.split(','))})
df_umap['CL'] = df_umap['cell_type'].map({'arterial': 'endothelial cell',
 'bcell': 'B cell',
 'chondrocytes': 'chondrocyte',
 'intermediate': 'fibroblast',
 'lining': 'fibroblast',
 'myeloid': 'macrophage',
 'neutrophil': 'neutrophil',
 'pericyte_VSMC': 'pericyte',
 'perivascular': 'fibroblast',
 'sublining': 'fibroblast',
 'tcell': 'T cell',
 'venous': 'endothelial cell'})

df_umap['batch'] = df_umap.apply(lambda row: '{status}_{label}'.format(**row), axis=1)
df_umap = df_umap.loc[:,'status,label,batch,cell_type,CL,umap_x,umap_y'.split(',')]

status
case    10418
ctrl     8073
Name: count, dtype: int64

label
KO     7762
WT     5429
ISO    3191
N3     2109
Name: count, dtype: int64

In [11]:
display(df_umap.head(2))
adata.obs = adata.obs.loc[:, []].join(df_umap)
adata.obs.head(2)

,status,label,batch,cell_type,CL,umap_x,umap_y
AACCGCGTCCGAATGT_BRI203,Healty,WT,Healty_WT,lining,fibroblast,2.419474,3.522746
GGGAATGAGGGTTCCC_BRI070,RA,anti-NRR3,RA_anti-NRR3,myeloid,macrophage,9.546767,2.541875


,status,label,batch,cell_type,CL,umap_x,umap_y
AAACCTGAGAACAATC_BRI203,Healty,WT,Healty_WT,perivascular,fibroblast,-6.669090,-1.876773
AAACCTGAGCTCCCAG_BRI203,Healty,WT,Healty_WT,sublining,fibroblast,-5.238616,-1.053888


In [12]:
display(type(adata.X))
adata.X = csr_matrix(adata.X)
display(type(adata.X))

scipy.sparse._csr.csr_matrix

scipy.sparse._csr.csr_matrix

In [14]:
# adata.write_h5ad('RA_GSE145286_mouse.h5ad')
# adata.obs.to_csv('RA_GSE145286_mouse_ann.csv',index=True)
h5ad_to_mtx(adata,'RA_GSE145286_mouse')

frist 10 data.X nonzero elements:
 [[1 1 1 1 2 2 1 1 1 1]]
[out] RA_GSE145286_mouse
